## Import Library

In [2]:
 %matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn;seaborn.set()
import scipy  
from scipy import stats
from sklearn.metrics import classification_report
# import scikits.bootstrap as bootstrap  


## Load, Transform, and EDA MRR

In [28]:
'''
Helper Class
'''
### Helper Display Class
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# Load rawMRR

rawMRR = pd.read_csv('../Data/MRR_Data_05242017.csv')
#for privacy reasons
rawMRR.drop(['Member_First_Name__c','Member_Last_Name__c'],axis=1).head()


,Chart_ID__c,Member_ID__c,MCC_Priority__c,Member_DOB__c,Member_Gender__c,FileBound_Status__c,FileBound_Disposition_Code__c,LocationKey,MRT_Name__c,ProviderFirstName,ProviderLastName,NPI
0,30024326514003113630,207_4_7438230_645369,A,1952-10-31 00:00:00.0000000,F,REL,QAP,L-00244110,Ciox (SFTP),ERNESTO,BONILLA MD,1508850918
1,30025145794003113677,207_4_7451059_647541,A,1964-12-08 00:00:00.0000000,M,REL,QAP,L-00245120,Ciox (SFTP),CHARLES,STRINGER PA,1457686404
2,30024273594003113264,207_4_9028261_3818634,A,1975-03-25 00:00:00.0000000,F,REL,QAP,L-00245454,SFTP,CHRISTOPHER,BORREGO MD,1811999543
3,30025141734003113271,207_6_10833337_432775,A,1971-10-25 00:00:00.0000000,F,REL,QAP,L-00245131,Ciox (SFTP),JOSHUA J,ANDERSON DO,1427284645
4,30024180394003113481,207_4_7438286_645411,A,1953-12-06 00:00:00.0000000,F,REL,QAP,L-00247266,Manny Diaz,JENNIFER A,ROMERO DO,1811966161


In [29]:
rawMRR.columns

Index(['Chart_ID__c', 'Member_ID__c', 'MCC_Priority__c',
       'Member_First_Name__c', 'Member_Last_Name__c', 'Member_DOB__c',
       'Member_Gender__c', 'FileBound_Status__c',
       'FileBound_Disposition_Code__c', 'LocationKey', 'MRT_Name__c',
       'ProviderFirstName', 'ProviderLastName', 'NPI'],
      dtype='object')

In [7]:
# add age column and transform  rawMRR, rename columns
rawMRR['age'] = rawMRR.Member_DOB__c.map(lambda date: 2017-int(date[:4]))
trfMRR = rawMRR.copy()[['Chart_ID__c','Member_ID__c','MCC_Priority__c','Member_Gender__c','LocationKey','NPI','age']]
trfMRR.rename(columns = {'Chart_ID__c':'ChartID','Member_ID__c':'MemberID',
                         'MCC_Priority__c':'Priority','Member_Gender__c':'Gender','age':'Age'},
                          inplace=True)
#trfMRR.set_index(['MemberID','ChartID'],inplace = True)

### MRR at Chart Level

At Chart Level, the features of interest are Age, Sex, Priority, # of Chart at Location, Specialty
So our Task is to computer the the # of charts at the location and normalize the Specialty

In [8]:
# Remind how trfMRR looks like
display('trfMRR.head()')

,ChartID,MemberID,Priority,Gender,LocationKey,NPI,Age
0,30024326514003113630,207_4_7438230_645369,A,F,L-00244110,1508850918,65
1,30025145794003113677,207_4_7451059_647541,A,M,L-00245120,1457686404,53
2,30024273594003113264,207_4_9028261_3818634,A,F,L-00245454,1811999543,42
3,30025141734003113271,207_6_10833337_432775,A,F,L-00245131,1427284645,46
4,30024180394003113481,207_4_7438286_645411,A,F,L-00247266,1811966161,64


In [9]:
#calculate chart count at location

chartCount_loc = trfMRR.copy()[['LocationKey','ChartID']]\
                 .groupby('LocationKey').count()\
                 .reset_index()
chartCount_loc.rename(columns={'ChartID':'Chart_Count'},inplace = True)
chart_MRR = trfMRR.copy()[['ChartID','Priority','Gender','Age','LocationKey','NPI']]
chart_MRR = pd.merge(chart_MRR,chartCount_loc,on='LocationKey',how='left')

print(trfMRR.shape,chart_MRR.shape)
display('chart_MRR.head()')

(641865, 7) (641865, 7)


,ChartID,Priority,Gender,Age,LocationKey,NPI,Chart_Count
0,30024326514003113630,A,F,65,L-00244110,1508850918,12548
1,30025145794003113677,A,M,53,L-00245120,1457686404,296
2,30024273594003113264,A,F,42,L-00245454,1811999543,145
3,30025141734003113271,A,F,46,L-00245131,1427284645,96
4,30024180394003113481,A,F,64,L-00247266,1811966161,285


#### import NPI data file and get NPI Specialty


In [10]:
rawNPI = pd.read_excel('../Aetna_NPI.xlsx',sheetname = 'NPI',header = 0)
rawNPI.head()

,NPI,name,desc,city,country_code,telephone_number,fax_number,state,postal_code,address_1,address_2,country_name,address_type,address_purpose
0,1003001017,NICHOLS LAWRENCE,Dermatology,LA PALMA,US,714-522-2041,714-522-8246,CA,906231745,5471 LA PALMA AVE,STE. 202,United States,DOM,LOCATION
1,1003001462,WOLSKI MICHAL,Radiology Radiation Oncology,GREENSBORO,US,706-454-1624,NaN,GA,306425285,1000 COWLES CLINC WAY,NaN,United States,DOM,LOCATION
2,1003002817,IGLESIAS NAYVIS,Family Medicine,DORAL,US,305-594-9333,305-594-0440,FL,331666658,3650 NW 82ND AVE,SUITE 502,United States,DOM,LOCATION
3,1003004540,CABANELLAS JENNINE,Specialist,PALM SPRINGS,US,561-227-3101,561-227-3182,FL,334613045,2889 10TH AVE N,SUITE 306,United States,DOM,LOCATION
4,1003005109,FETTMAN NICHOLAS,Otolaryngology,OXNARD,US,805-983-0395,805-983-0463,CA,930303790,1700 N ROSE AVE,SUITE 460,United States,DOM,LOCATION


In [11]:
'''Get the description of specialties for manual normalization in Excel '''

UniqSpec = rawNPI['desc'].unique().tolist()
print(len(UniqSpec), ' specialties')


# export CSV for manual editing
rawNPI.groupby('desc').count()['NPI'].to_csv('groupedNPI.csv')

502  specialties


In [15]:
'''import normNPI'''

renamedNPI = pd.read_csv('renamedNPI.csv')
lookupDICT = dict(zip(renamedNPI['Original'],renamedNPI['New']))
lookupDICT
normNPI = rawNPI.copy()['desc'].map(lookupDICT)
trfNPI = rawNPI.copy()
trfNPI['Specialty'] = normNPI
#trfNPI = rawNPI.copy().replace({'desc':lookupDICT},inplace=True)

NPI_Specialty = trfNPI.copy()[['NPI','Specialty','state']]
display('trfNPI.head()','NPI_Specialty.head()')

trfNPI.head()
          NPI                name                          desc          city  \
0  1003001017    NICHOLS LAWRENCE                   Dermatology      LA PALMA   
1  1003001462       WOLSKI MICHAL  Radiology Radiation Oncology    GREENSBORO   
2  1003002817     IGLESIAS NAYVIS               Family Medicine         DORAL   
3  1003004540  CABANELLAS JENNINE                    Specialist  PALM SPRINGS   
4  1003005109    FETTMAN NICHOLAS                Otolaryngology        OXNARD   

  country_code telephone_number    fax_number state postal_code  \
0           US     714-522-2041  714-522-8246    CA   906231745   
1           US     706-454-1624           NaN    GA   306425285   
2           US     305-594-9333  305-594-0440    FL   331666658   
3           US     561-227-3101  561-227-3182    FL   334613045   
4           US     805-983-0395  805-983-0463    CA   930303790   

               address_1  address_2   country_name address_type  \
0      5471 LA PALMA AVE   STE. 202  United States          DOM   
1  1000 COWLES CLINC WAY        NaN  United States          DOM   
2       3650 NW 82ND AVE  SUITE 502  United States          DOM   
3        2889 10TH AVE N  SUITE 306  United States          DOM   
4        1700 N ROSE AVE  SUITE 460  United States          DOM   

  address_purpose        Specialty  
0        LOCATION      Dermatology  
1        LOCATION        Radiology  
2        LOCATION  Family Medicine  
3        LOCATION       Specialist  
4        LOCATION   Otolaryngology  

NPI_Specialty.head()
          NPI        Specialty state
0  1003001017      Dermatology    CA
1  1003001462        Radiology    GA
2  1003002817  Family Medicine    FL
3  1003004540       Specialist    FL
4  1003005109   Otolaryngology    CA

#### Join NPI to get Feature Matrix for Chart Level Analaysis

In [16]:
display('chart_MRR.head()','NPI_Specialty.head()')

chart_MRR.head()
                ChartID Priority Gender  Age LocationKey         NPI  \
0  30024326514003113630        A      F   65  L-00244110  1508850918   
1  30025145794003113677        A      M   53  L-00245120  1457686404   
2  30024273594003113264        A      F   42  L-00245454  1811999543   
3  30025141734003113271        A      F   46  L-00245131  1427284645   
4  30024180394003113481        A      F   64  L-00247266  1811966161   

   Chart_Count  
0        12548  
1          296  
2          145  
3           96  
4          285  

NPI_Specialty.head()
          NPI        Specialty state
0  1003001017      Dermatology    CA
1  1003001462        Radiology    GA
2  1003002817  Family Medicine    FL
3  1003004540       Specialist    FL
4  1003005109   Otolaryngology    CA

In [17]:
#Check how many NPI are retrieved are in MRR
print('size of NPI Conversion',NPI_Specialty.shape)
NPI_MRR = trfMRR['NPI'].unique().tolist()
# Check if
NPI_API = NPI_Specialty['NPI'].unique().tolist()
commonID = list(set(NPI_MRR).intersection(NPI_API))
print( 'Unique NPI from MRR:',len(NPI_MRR),\
       '\nUnique NPI count from API:',len(NPI_API),\
       '\nCommon member from MCC and MRR:', len(commonID))

size of NPI Conversion (109072, 3)
Unique NPI from MRR: 111382 
Unique NPI count from API: 109072 
Common member from MCC and MRR: 109072


In [18]:
# Left Join to put NPI info on MRR
chart_MRR = pd.merge(chart_MRR,NPI_Specialty,on = 'NPI',how='left')

print(chart_MRR[chart_MRR['Specialty'].isnull()].shape)

chart_MRR = chart_MRR.dropna().drop(['LocationKey','NPI'],axis=1)
print(chart_MRR.shape)
display('chart_MRR.head()')

(14717, 9)
(627148, 7)


,ChartID,Priority,Gender,Age,Chart_Count,Specialty,state
0,30024326514003113630,A,F,65,12548,Family Medicine,FL
1,30025145794003113677,A,M,53,296,Physician Assistant,FL
2,30024273594003113264,A,F,42,145,Internal Medicine,GA
3,30025141734003113271,A,F,46,96,Family Medicine,IA
4,30024180394003113481,A,F,64,285,Family Medicine,FL


In [32]:
# number of distinct Specialty

len(chart_MRR.Specialty.unique())

57

## Load and EDA MCC

In [19]:
rawMCC = pd.read_csv('../data/MCC_Data_05242017.csv')
rawMCC.head()

,MemberID,ChartID,Priority,FlexField6,PageCount,BeginDos,ENDDos,DxCode,ClaimsNumber,ClaimsComment,ClaimsResults,HCCCode
0,101_101905214,30040973694004696467,NaN,A,2.0,2016-01-18,2016-01-18,-,103_EPY0RM9JG_00,Valid,Added from claims,NaN
1,101_102316015,30040673064004666404,NaN,B,19.0,2016-07-08,2016-07-08,000,NaN,NaN,Exclude,0.0
2,101_102316015,30040673064004666404,NaN,B,19.0,2016-07-08,2016-07-08,I472,103_ENPBTZWQD_00,ASR,Added from claims,NaN
3,101_102316015,30040673064004666404,NaN,B,19.0,2016-08-30,2016-08-30,000,NaN,NaN,Exclude,0.0
4,101_103164910,30025048294003103927,NaN,A,15.0,2016-03-02,2016-03-02,000,NaN,NaN,Exclude,0.0


In [20]:
rawMCC.columns

Index(['MemberID', 'ChartID', 'Priority', 'FlexField6', 'PageCount',
       'BeginDos', 'ENDDos', 'DxCode', 'ClaimsNumber', 'ClaimsComment',
       'ClaimsResults', 'HCCCode'],
      dtype='object')

### Transform MCC

In [21]:
# Select relevant MCC columns for assessing HCC
trfMCC = rawMCC.copy()\
               .loc[:,['MemberID','ChartID','ClaimsComment','ClaimsResults','HCCCode']]
display('trfMCC.head()')


,MemberID,ChartID,ClaimsComment,ClaimsResults,HCCCode
0,101_101905214,30040973694004696467,Valid,Added from claims,NaN
1,101_102316015,30040673064004666404,NaN,Exclude,0.0
2,101_102316015,30040673064004666404,ASR,Added from claims,NaN
3,101_102316015,30040673064004666404,NaN,Exclude,0.0
4,101_103164910,30025048294003103927,NaN,Exclude,0.0


In [22]:
'''The chart is unique in MCC at the Dx code level
so when the chosen columns are Claim Comment ClaimResults and HCCCode
there will be duplicates at ChartID and HCCCode, these can be consolidated'''

#look in columns ChartID and HCCCode to determine duplicate
trfMCC = trfMCC.drop_duplicates(['ChartID','HCCCode'],keep='first')

# An example same MemberID different HCCCode
trfMCC[trfMCC['ChartID'] == '30024121804003011278']

,MemberID,ChartID,ClaimsComment,ClaimsResults,HCCCode
1150323,207_4_807745_1007741,30024121804003011278,Valid,Added from claims,NaN
1150324,207_4_807745_1007741,30024121804003011278,Add,No Match in Claims,20.0
1523501,207_4_807745_1007741,30024121804003011278,NaN,Exclude,0.0


In [23]:
'''Since this analysis deals with HCC vs no HCC 
we drop NaN from the HCC column'''
print(trfMCC['ChartID'].shape)
trfMCC = trfMCC[np.isfinite(trfMCC['HCCCode'])]
display('trfMCC.head(10)')

(1414891,)


,MemberID,ChartID,ClaimsComment,ClaimsResults,HCCCode
1,101_102316015,30040673064004666404,NaN,Exclude,0.0
4,101_103164910,30025048294003103927,NaN,Exclude,0.0
5,101_103164910,30025048304003103928,NaN,Exclude,0.0
10,101_103512416,30024883134003803001,AddCC,No Match in Claims,47.0
12,101_103564476,30024260684003400829,NaN,Exclude,0.0
13,101_103564476,30024539994003400828,NaN,Exclude,0.0
15,101_103564476,30028017294003400827,NaN,Exclude,0.0
16,101_103564495,30024539994003400856,NaN,Exclude,10.0
17,101_103704717,30033166684003915766,Add,No Match in Claims,142.0
18,101_103704717,30033166714003915769,Add,No Match in Claims,142.0


#### Chart Level MCC

In [36]:

"""
# code reseved to count the # of HCC found by EpiSource
countHCC = chart_MCC.query("ClaimsComment == 'AddCC' or ClaimsComment =='Add' or ClaimsComment =='ASR' and ClaimsResults !='Exclude' and HCCCode > 0")
countHCC = countHCC.groupby('ChartID').count()['HCCCode'].reset_index()
print(countHCC.shape)
display('chart_MCC.head()','countHCC.head()')
"""

'\n# code reseved to count the # of HCC found by EpiSource\ncountHCC = chart_MCC.query("ClaimsComment == \'AddCC\' or ClaimsComment ==\'Add\' or ClaimsComment ==\'ASR\' and ClaimsResults !=\'Exclude\' and HCCCode > 0")\ncountHCC = countHCC.groupby(\'ChartID\').count()[\'HCCCode\'].reset_index()\nprint(countHCC.shape)\ndisplay(\'chart_MCC.head()\',\'countHCC.head()\')\n'

In [25]:
''' Find which chart has HCC, return TRUE, else False'''
chart_MCC = trfMCC.copy().iloc[:,1:]

chart_MCC = chart_MCC.groupby('ChartID').sum()['HCCCode'].reset_index()

chart_MCC['hasHCC'] = chart_MCC['HCCCode'] >0
chart_MCC.drop('HCCCode',axis=1,inplace =True)



# Merging MRR and MCC

In [26]:
display('chart_MRR.head()','chart_MCC.head()')

chart_MRR.head()
                ChartID Priority Gender  Age  Chart_Count  \
0  30024326514003113630        A      F   65        12548   
1  30025145794003113677        A      M   53          296   
2  30024273594003113264        A      F   42          145   
3  30025141734003113271        A      F   46           96   
4  30024180394003113481        A      F   64          285   

             Specialty state  
0      Family Medicine    FL  
1  Physician Assistant    FL  
2    Internal Medicine    GA  
3      Family Medicine    IA  
4      Family Medicine    FL  

chart_MCC.head()
                ChartID  hasHCC
0  30024121804003011278    True
1  30024121804003045502    True
2  30024121804003061766    True
3  30024121804003094366   False
4  30024121804003153900   False

In [27]:
HCC = pd.merge(chart_MRR,chart_MCC,on='ChartID',how='inner')
print(HCC.shape)
display('HCC.head(10)')

(571892, 8)


,ChartID,Priority,Gender,Age,Chart_Count,Specialty,state,hasHCC
0,30024326514003113630,A,F,65,12548,Family Medicine,FL,True
1,30025145794003113677,A,M,53,296,Physician Assistant,FL,False
2,30024273594003113264,A,F,42,145,Internal Medicine,GA,False
3,30025141734003113271,A,F,46,96,Family Medicine,IA,False
4,30024180394003113481,A,F,64,285,Family Medicine,FL,True
5,30025133394003112437,A,F,59,35,Physician Assistant,NC,True
6,30024273594003112465,B,F,53,145,Internal Medicine,GA,True
7,30025137514003112849,B,F,48,9,Institution,WA,False
8,30025147874003113885,A,M,58,296,Family Medicine,FL,True
9,30024184914003113886,A,M,58,59,Specialist,FL,False


In [42]:
# export CSV for manual editing
HCC.to_csv('featureMatrix.csv')